In [141]:
# load the python library
import polars as pl 

# set the configuration to see 100 lines instead of truncated set
pl.Config.set_tbl_rows(100)

polars.config.Config

In [142]:
#create a variable with the file path
file = 'pranic_study_data_original.csv'

# clean the column names
clean_columns = (pl
                 .all() # all columns
                 .name # namespace for a set of column functions
                 .map(lambda col: col
                      .strip() # remove leading and trailing white space
                      .lower() # set column names to lower string
                      .replace('/', '_') # replace slash with underscore
                      .replace('-', '_') # replace hyphen with underscore
                      .replace(' ', '_') # replace empty with underscore
                      .replace('+', '_') # replace addition with underscore
                      .replace('@', '') # replace ampersand with underscore
                      )
                 )


# dictionary to rename columns
rename_column_dictionary = {'part_no':'subject_id',
                            'c_i':'group',
                            'defo':'deformity',
                            'dia_since':'duration',
                            'hba1c_fastg_d1':'hba1c_start',
                            'hba1c_d35_':'hba1c_end',
                            '%_change':'hba1c_pct_change',
                            '_duplicated_0':'hba1c_abs_change',
                            'd1':'wagner_pre',
                            'd35_':'wagner_post',
                            'imp_no':'wagner_post_status',
                            'vib':'vpt',
                            '_d1':'wound_area_start', 
                            '_d_35_40':'wound_area_end', 
                            '_duplicated_1':'wound_abs_change',
                            'd_35_40':'wound_pct_change',
                            '_duplicated_2':'well_being_score',
                            'well_being':'well_being_change'
                            }


# add leading zeros, such that all values are 3 charaters long
zfill_columns = pl.col.subject_id.str.zfill(3)

# rename groups
rename_group = pl.col.group.replace({'I':'treatment', 'C':'SOC'})

# convert datatype to boolean for enumerated columns
convert_to_boolean = (pl
                      .col(['deformity', 'on_insulin', 'heart_conditions', 'vericose'])
                      .str.to_lowercase()
                      .str.strip_chars()
                      .replace({'no':0, 'yes':1})
                      .cast(pl.Int16)
                      .cast(pl.Boolean)
                      )

# convert datatype to integer for columns
convert_to_integer = pl.col(['duration', 'age']).cast(pl.Int16)

# convert datatype to float for enumerated columns
convert_to_float = (pl
                    .col(['hba1c_start', 'hba1c_end', 'hba1c_pct_change', 'hba1c_abs_change',
                          'wound_area_start', 'wound_area_end', 'wound_abs_change', 'wound_pct_change', 'well_being_score'])
                    .str.replace('%', '')
                    .cast(pl.Float32)
                    )

# change certain values to null
fix_wagner_post = (pl
              .when(pl.col.wagner_post.is_in(['#REF!', "NA"]))
              .then(None)
              .otherwise(pl.col.wagner_post)
              .alias('wagner_post')
              )

# clean up wagner column
fix_wagner_post_status = (pl
                          .col.wagner_post_status
                          .replace({'yes':'improved', 'sq':'no_change', None:'declined'})
                          )

# clean up mono column
fix_mono = (pl
            .when(pl.col.mono.is_in(["NA"]))
            .then(None)
            .otherwise(pl.col.mono.str.to_lowercase().str.strip_chars().replace({'sq':'no_change', 'impr':'improved'}))
            .alias('mono')
            )

# clean up vpt column
fix_vpt = (pl
            .when(pl.col.vpt.is_in(["NA"]))
            .then(None)
            .otherwise(pl.col.vpt.str.to_lowercase().str.strip_chars().replace({'sq':'no_change', 'impr':'improved'}))
            .alias('vpt')
            )

# clean up well-being column
fix_well_being_change = (pl
                        .when(pl.col.well_being_change.is_in(["NA"]))
                        .then(None)
                        .otherwise(pl.col.well_being_change.str.to_lowercase().str.strip_chars().replace({'sq':'no_change', 'impr':'improved', 'imp':'improved'}))
                        .alias('well_being_change')
                        )

# recalculate wound area absolute and percentage change
calculate_wound_abs_change = pl.col.wound_area_end.sub(pl.col.wound_area_start).alias('wound_abs_change')
calculate_wound_pct_change = (pl.col.wound_area_end.sub(pl.col.wound_area_start)).truediv(pl.col.wound_area_start).alias('wound_pct_change')

# recalculate a1c absolute and percentage change
calculate_hba1c_abs_change = pl.col.hba1c_end.sub(pl.col.hba1c_start).alias('hba1c_abs_change')
calculate_hba1c_pct_change = (pl.col.hba1c_end.sub(pl.col.hba1c_start)).truediv(pl.col.hba1c_start).alias('hba1c_pct_change')


#load data and run cleaning functions
data = (pl
        .read_csv(file, infer_schema_length=0) # load the csv text file, and tell polars not to guess datatypes
        .select(clean_columns) # clean the column names
        .filter(pl.col.defo.is_not_null()) # remove null rows
        .drop('', '_duplicated_3', '_duplicated_4', 'fpbs_d1', 'fbbs_end') # null or unnecessary columns
        .rename(rename_column_dictionary) # rename the columns
        .with_columns(zfill_columns, rename_group, convert_to_boolean, convert_to_integer, convert_to_float, fix_wagner_post, fix_wagner_post_status) # run cleaning functions from above
        .with_columns(fix_mono, fix_vpt, fix_well_being_change) # run cleaning functions from above
        .with_columns(calculate_wound_abs_change, calculate_wound_pct_change, calculate_hba1c_abs_change, calculate_hba1c_pct_change) # run cleaning functions from above
        )

data.head()

subject_id,group,deformity,gender,age,on_insulin,heart_conditions,vericose,duration,hba1c_start,hba1c_end,hba1c_pct_change,hba1c_abs_change,wagner_pre,wagner_post,wagner_post_status,mono,vpt,wound_area_start,wound_area_end,wound_abs_change,wound_pct_change,well_being_score,well_being_change
str,str,bool,str,i16,bool,bool,bool,i16,f32,f32,f32,f32,str,str,str,str,str,f32,f32,f32,f32,f32,str
"""001""","""treatment""",false,"""M""",55,true,false,false,15,12.3,10.9,-0.113821,-1.400001,"""2""","""1""","""improved""","""no_change""","""no_change""",6.45,0.0,-6.45,-1.0,80.0,"""improv"""
"""002""","""treatment""",false,"""M""",64,true,true,false,25,11.8,12.9,0.09322,1.099999,"""2""","""2""","""no_change""","""no_change""","""no_change""",0.74,0.12,-0.62,-0.837838,0.0,"""no_change"""
"""004""","""treatment""",false,"""M""",44,false,false,true,8,12.3,9.6,-0.219512,-2.7,"""2""","""1""","""improved""","""no_change""","""improved""",5.03,2.16,-2.87,-0.570577,100.0,"""improv"""
"""005""","""treatment""",false,"""F""",52,false,false,true,30,8.5,7.8,-0.082353,-0.7,"""2""","""1""","""improved""","""no_change""","""improved""",10.5,5.54,-4.96,-0.472381,100.0,"""improv"""
"""010""","""treatment""",false,"""M""",68,true,false,false,30,null,null,null,null,"""1""",null,"""improved""","""worse""","""worse""",0.21,0.0,-0.21,-1.0,100.0,"""improv"""


In [143]:
data.write_excel('pranic_study_data_cleaned.xlsx')
data.write_parquet('pranic_study_data_cleaned.parquet')